In [ ]:
from block_ai.lib.myblokus import point
from block_ai.lib.myblokus.game_engine import GameEngine
from block_ai.lib.myblokus.game import Game, GameEnd
from block_ai.lib.myblokus.board import Board
from block_ai.lib.myblokus.point import Point
from block_ai.lib.myblokus import point
from block_ai.lib.myblokus.piece import Piece
from block_ai.lib.myblokus.corner import Corner
from block_ai.lib.myblokus.orientation import Orientation
from block_ai.lib.myblokus.move import Move
from block_ai.lib.myblokus.agent import HumanInput, RandomAgent, Agent, PointAgent
from block_ai.lib.myblokus.board_view import BoardView


import itertools
import random
from copy import deepcopy
import numpy as np

from tqdm import tqdm_notebook

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

%load_ext line_profiler

import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
ge = GameEngine(display=False, players=['greedy', 'greedy', 'greedy', 'greedy'])
ge.play_game()

Game upgrades:
    moves contains duplicates
        dict of dicts vs dict with tuple of (corner, piece) key. latter's probably faster

    Is it really important to get all with a corner or all with a piece fast? maybe cut one out to speed up overall
  

Performance Tuning:

1. Write Game Test compare valid 

In [ ]:
def get_avg_moves_per_game():
    
    products = []

    for i in range(100):

        if i > 0 and i % 10 == 0:
            print(i)

        product = get_moves_per_game()
        products.append(product)
        
def get_moves_per_game():
    ge = GameEngine()
    move_counts = []
    while True:
        try:
            player_id = ge.game.player_pointer
            moves = list(ge.game.get_players_moves(player_id))
            move_counts.append((player_id, len(moves)))
            ge.play_turn()

        except GameEnd:
            break

    ids, options = zip(*move_counts)
    options = [float(o) for o in options]
    product = np.product(options)
    return product

In [ ]:
def show_orientation(o):
    """displays an orientation"""
    board = Board()
    vis = BoardView()
    offset = Point(10, 10)

    for i, j in itertools.product(range(20), range(20)):
        p = Point(i, j)
        val = -1
        if i < 10:
            if j < 10:
                val = 1
            else:
                val = 0
        else:
            if j < 10:
                val = 2
        board.assign(p, val)
            
    corners = o.get_corners()

    for c in corners:
        board.assign(c.p2 + offset, 4)
    new_points = [p + offset for p in o.points]
    for p in new_points:
        board.assign(p, 3)

    board.assign(Point(-1, -1), 6)
    board.display()

50% of get corner moves involves rotating points. Are matracies faster?

Nagging Ideas:
     -  I should set the seeds for random players to or replay a game based on move history
        so that the runtimes are the same
    
     -  write tests so I don't break anything making optimizations

     -  Speed up BoardViewer 

Optimization Ideas:
     - lots of runtime spend rotating points. Are np.arrays / rot methods faster?
       Answer: No
     
     - a bunch of time is spent converting orientations to sets in player.validate_move
         - raise try / catch is pretty expensive though it's nice to have reason why move is invalid
         - could use a boolean check + explain menthod (a little ugly)
  
     - logging can slow down stuff remove ones from the inner most loops

## Status
Refactored but a bunch of cornsers get droped for no reason near the end

#### Questions:

why does it allow invalid moves in the below loop?

Theres an average branching factor of around 25
so call it 75 is that doable?





Number of chess game one guy says with 40 avg moves avg 30 moves per game
~ 10 ** 120 

James Hardy says (10 ** 50) ** 50 

Number of blockus game???

according to the game engine there are ~10^140 move options